In [2]:
import pandas as pd
from data.data import Data

ROWS = 200 # Limit 6000

data_raw = Data().get_all_data()


# 8670 ROWS INPUT #
df1 = data_raw['twitter_MBTI']
df1 = df1[['label', 'text']]
df1.rename(columns={'label': 'type'}, inplace=True)
df1 = df1.sample(n = ROWS, ignore_index = True)

# 106062 ROWS INPUT #
df2 = data_raw['MBTI 500']
df2 = df2[['type', 'posts']]
df2.rename(columns={'posts': 'text'}, inplace=True)
df2 = df2.sample(n = ROWS, ignore_index = True) # > 15000 combined, > 20000 df1 + df2

## SPECIFCALLY CREATING DIV 0 ERROR, ASK ABOUT IT ##
# 7811 ROWS INPUT #
df3 = data_raw['mbti_1']
df3.rename(columns={'posts': 'text'}, inplace=True)
df3 = df3.iloc[:1000]
# df3 = df3.sample(n = ROWS, ignore_index = True)
# ####################################################

## Combined all data for PP if desired
data_combined = pd.concat([df1, df2, df3], ignore_index=True)
data_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    1400 non-null   object
 1   text    1400 non-null   object
dtypes: object(2)
memory usage: 22.0+ KB


/tmp/ipykernel_11761/635315286.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'label': 'type'}, inplace=True)


In [3]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize

df_pp = training_preprocessing(data_combined)
df_pp_os = training_oversampling(df_pp)
df_pp_os_bal = training_balancing(df_pp_os)
df_list = training_vectorize(df_pp_os_bal)


Dataset contains 7811 rows

cleaned dataset contains 1400 rows and 8 columns
E-I dataset contains 734 rows and 2420 columns
S-N dataset contains 398 rows and 2514 columns
F-T dataset contains 1392 rows and 2375 columns
P-J dataset contains 1138 rows and 2378 columns


In [4]:
from model.model import *

model_list = initialize_models(new_models = True)
model_list 

[]

In [5]:
param_list = grid_search_all_models(df_list)
param_list 

Model Searched = SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000) IE
Best parameters: {'alpha': 0.0001, 'penalty': 'elasticnet'}
Best score: 0.6315248429468875
Model Searched = SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000) NS
Best parameters: {'alpha': 0.0001, 'penalty': 'l1'}
Best score: 0.7625974025974026
Model Searched = SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000) FT
Best parameters: {'alpha': 0.0001, 'penalty': 'l1'}
Best score: 0.7022257467618293
Model Searched = SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000) JP
Best parameters: {'alpha': 0.001, 'penalty': 'elasticnet'}
Best score: 0.5314308176100628


[{'alpha': 0.0001, 'penalty': 'elasticnet'},
 {'alpha': 0.0001, 'penalty': 'l1'},
 {'alpha': 0.0001, 'penalty': 'l1'},
 {'alpha': 0.001, 'penalty': 'elasticnet'}]

In [6]:
model_list = initialize_models(params = param_list)
model_list

[SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000,
               penalty='elasticnet'),
 SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1'),
 SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1'),
 SGDClassifier(alpha=0.001, early_stopping=True, loss='log_loss', max_iter=5000,
               penalty='elasticnet')]

In [7]:
history = train_model(df_list, model_list)

F1-score: 52.723%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000,
              penalty='elasticnet')
MBTI Type: IE
F1-score: 65.278%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1')
MBTI Type: NS
F1-score: 58.544%
Model: SGDClassifier(early_stopping=True, loss='log_loss', max_iter=5000, penalty='l1')
MBTI Type: FT
F1-score: 49.208%
Model: SGDClassifier(alpha=0.001, early_stopping=True, loss='log_loss', max_iter=5000,
              penalty='elasticnet')
MBTI Type: JP


In [8]:
df = data_raw['MBTI 500']
df = df[['type', 'posts']]
df.rename(columns={'posts': 'text'}, inplace=True)
df = df.sample(n = 10, ignore_index = True)

estimation = df['type'].iloc[:1].tolist()[0]
df_pred = df.iloc[:1].drop(columns =['type'])

estimation

'ENTJ'

In [9]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize

df_pred_pp = prediction_preprocessing(df_pred)
df_pred_list = prediction_vectorize(df_pred_pp)

In [10]:
predict_model(texts = df_pred_list, verbose = False)
print(f"Actual = {estimation}")

Final Prediction = ESTP
Actual = ENTJ
